In [2]:
#! pip install autots
#! pip install  tensorflow
#! pip install --no-cache-dir -I pystan==2.19.1.1
#! pip3 install fbprophet --no-cache-dir --no-binary :all:


In [3]:
#from tensorflow.keras.layers import Dense, Dropout, LSTM
#from tensorflow.keras.models import Sequential

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn import regression
sns.set()
plt.style.use('seaborn-whitegrid')
import warnings
warnings.filterwarnings('ignore')
import datetime
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go

#from tensorflow.python.keras.layers import Dense, Dropout, LSTM
#from tensorflow.python.keras.models import Sequential

In [5]:
df_adj_close = pd.read_csv(r'../data/adj_close.csv')
df_close = pd.read_csv(r'../data/close.csv')
df_high = pd.read_csv(r'../data/high.csv')
df_low = pd.read_csv(r'../data/low.csv')
df_open = pd.read_csv(r'../data/open.csv')
df_volume = pd.read_csv(r'../data/volume.csv')

In [6]:
datas = []
crypto_list = ['ADA-USD', 'ATOM-USD', 'AVAX-USD', 'AXS-USD', 'BTC-USD','ETH-USD', 'LINK-USD', 'LUNA1-USD', 'MATIC-USD', 'SOL-USD']
for crypto in crypto_list: 
    data = {'Date': df_close['Date'],
        'adj_close': df_adj_close[crypto],
        'close': df_close[crypto],
        'low': df_low[crypto],
        'high': df_high[crypto],
        'open': df_open[crypto],
        'volume': df_volume[crypto]
        }
    
    datas.append(data)

df_ada = pd.DataFrame(datas[0]).dropna()
df_atom= pd.DataFrame(datas[1]).dropna()
df_avax= pd.DataFrame(datas[2]).dropna()
df_axs= pd.DataFrame(datas[3]).dropna()
df_btc= pd.DataFrame(datas[4]).dropna()
df_eth= pd.DataFrame(datas[5]).dropna()
df_link= pd.DataFrame(datas[6]).dropna()
df_luna1= pd.DataFrame(datas[7]).dropna()
df_matic= pd.DataFrame(datas[8]).dropna()
df_sol= pd.DataFrame(datas[9]).dropna()

In [7]:
df_btc['Date'] = pd.to_datetime(df_btc['Date']).dt.date

# Train Test Split

In [8]:
def train_test_split_time_series(data, num_df, column_date):
#data: data frame to split
#num_df: number of differente splits
#column_date: name of the column that has the Date info

    date_split=[]
    max_date = data[column_date].max()
    min_date = data[column_date].min()
    range_date = max_date-min_date
    split_criteria = 0.2/(num_df-1)
    train = 0.6
    test = 0.8
    inicial = 0
    for i in range(num_df):
        list_to_add=[]
        list_to_add.append((min_date + range_date*inicial))
        list_to_add.append((min_date + range_date*train))
        list_to_add.append((min_date + range_date*test))
        train = train + split_criteria
        test = test + split_criteria
        inicial = inicial + split_criteria

        date_split.append(list_to_add)
    
    
    return date_split



In [9]:
date = train_test_split_time_series(df_btc, num_df=3, column_date='Date')

In [10]:
date

[[datetime.date(2017, 4, 26),
  datetime.date(2020, 4, 25),
  datetime.date(2021, 4, 25)],
 [datetime.date(2017, 10, 25),
  datetime.date(2020, 10, 24),
  datetime.date(2021, 10, 24)],
 [datetime.date(2018, 4, 26),
  datetime.date(2021, 4, 25),
  datetime.date(2022, 4, 25)]]

In [11]:
train_1 = df_btc[(df_btc['Date'] >= date[0][0]) & (df_btc['Date'] < date[0][1])]
test_1 = df_btc[(df_btc['Date'] >= date[0][1]) & (df_btc['Date'] < date[0][2])]

train_2 = df_btc[(df_btc['Date'] >= date[1][0]) & (df_btc['Date'] < date[1][1])]
test_2 = df_btc[(df_btc['Date'] >= date[1][1]) & (df_btc['Date'] < date[1][2])]

train_3 = df_btc[(df_btc['Date'] >= date[2][0]) & (df_btc['Date'] < date[2][1])]
test_3 = df_btc[(df_btc['Date'] >= date[2][1]) & (df_btc['Date'] < date[2][2])]

In [12]:
metric_features = ['adj_close', 'close', 'low', 'high', 'open', 'volume']

# Normalization

In [13]:
scaler = MinMaxScaler()
train_1_min_max = scaler.fit_transform(train_1[metric_features])
test_1_min_max = scaler.transform(test_1[metric_features])

In [14]:
df_train = train_1[["Date", "high"]]
new_names = {
    "Date": "ds", 
    "high": "y",
}
df_train.rename(columns=new_names, inplace=True)

df_test = test_1[["Date", "high"]]
new_names = {
    "Date": "ds", 
    "high": "y",
}
df_test.rename(columns=new_names, inplace=True)

In [15]:
# plot the open price
x = df_train["ds"]
y = df_train["y"]
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y))
# Set title
fig.update_layout(
    title_text="Time series plot of BitCoin High Price (Train)",
)

In [16]:
# plot the open price
x = df_test["ds"]
y = df_test["y"]
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y))
# Set title
fig.update_layout(
    title_text="Time series plot of BitCoin High Price (Test)",
)